# Phase 2: Break It

In [201]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

# install checklist
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb

#!python3 -m spacy download en_core_web_sm
# install spacy
import spacy

import json

In [202]:
TRAIN = '../data/interim/train.csv'

In [203]:
data_train = pd.read_json('../data/raw/music_reviews_train.json.gz', lines=True)
data_train['sentiment']

0        positive
1        negative
2        negative
3        positive
4        positive
           ...   
99995    positive
99996    negative
99997    positive
99998    negative
99999    negative
Name: sentiment, Length: 100000, dtype: object

In [204]:
labels = data_train['sentiment'].tolist()

In [205]:
input_data = []
label = [] 
c = 0
n = 100 # set how many training examples to use.
with open(TRAIN, mode='r') as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
        if c < n:
            input_data.append(lines[0])
            label.append(int(lines[1]))
            c+=1
        else:
            break

In [206]:
def tkn(sentence):
    """Function to find all tokens in a given sentence
    """
    tok = re.compile('[\'\"]|[A-Za-z]+|[.?!:\'\"]+')
    
    return tok.findall(sentence)
    
def splitsies(para):
    punct = re.compile('[.?!:]')
    t = punct.split(para)
    spl= []
    for i in t:
        temp = tkn(i)
        if len(temp) > 0:
            spl.append(temp)
        
    return spl
    
#splitsies(input_data[0])

In [207]:
print(f'Sentence 1:\n{input_data[0]}\nLabel: {label[0]}')
type(input_data)

Sentence 1:
Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on this CD. I should have bought it years ago!
Label: 1


list

In [208]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

token_text = []
for para in input_data:
    token_text.append(sent_tokenize(para))
token_text[0]

[nltk_data] Downloading package punkt to /home/dmdequin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['Gotta listen to this!',
 'So creative!',
 'Love his music - the words, the message!',
 'Some of my favorite songs on this CD.',
 'I should have bought it years ago!']

In [209]:
nlp = spacy.load('en_core_web_sm')

# Add Typos

In [210]:
ret = Perturb.perturb(input_data, Perturb.add_typos)
ret.data[0]

['Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on this CD. I should have bought it years ago!',
 'Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on this CD. I shoul dhave bought it years ago!']

In [211]:
# This just typos one sentence in input paragraph
typo = []
for thing in ret.data:
    typo.append(thing[1])
typo[0]
#typo

'Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on this CD. I shoul dhave bought it years ago!'

In [212]:
# If "sentence" in para is only 1 char, add ".." to avoid errors
for i in range(len(token_text)):
    for j in range(len(token_text[i])):
        if len(token_text[i][j]) <= 1:
            token_text[i][j] = token_text[i][j]+'..'
            
token_text[6]

['Five Stars I love all of his music!', '!..']

In [213]:
# Typo every sentence in every paragraph
typoed = []
lala = []
for i in range(len(token_text)):
    ret = Perturb.perturb(token_text[i], Perturb.add_typos)
    typos = []
    for sent in ret.data:
        typos.append(sent[1])
        lala.append(sent[1])
    typoed.append(typos)
typoed[6]
#typoed

['Five Stars I love all ofh is music!', '!..']

In [214]:
# Convert back to list of paragraphs WOWWWW
tp = []
for i in range(len(typoed)):
    para = ""
    for j in range(len(typoed[i])):
        para = para + typoed[i][j] + " "
    tp.append(para)
#tp

# POS tag data

https://spacy.io/usage/processing-pipelines

When you call nlp on a text, spaCy will tokenize it and then call each component on the Doc, in order. It then returns the processed Doc that you can work with.

In [215]:
pdata = list(nlp.pipe(tp))
#for doc in nlp.pipe(input_data):
    # Do something with the doc here
#    print([(ent.text, ent.label_) for ent in doc.ents])

# Remove End Punctuation

In [216]:
pdata[0], Perturb.strip_punctuation(pdata[0])

(Gotta listent o this! So craetive! Love his msuic - the words, the message! Some of my favorite songs ont his CD. I sohuld have bought it years ago! ,
 'Gotta listent o this! So craetive! Love his msuic - the words, the message! Some of my favorite songs ont his CD. I sohuld have bought it years ago')

In [217]:
ret = Perturb.perturb(pdata, Perturb.punctuation)
no_punct = []
for i in ret.data:
    no_punct.append(i[1])
no_punct[0]
len(no_punct)
no_punct[0]

'Gotta listent o this! So craetive! Love his msuic - the words, the message! Some of my favorite songs ont his CD. I sohuld have bought it years ago'

# Negation

In [218]:
no_punct[0]

'Gotta listent o this! So craetive! Love his msuic - the words, the message! Some of my favorite songs ont his CD. I sohuld have bought it years ago'

In [219]:
pdata = list(nlp.pipe(no_punct))
pdata[0]

Gotta listent o this! So craetive! Love his msuic - the words, the message! Some of my favorite songs ont his CD. I sohuld have bought it years ago

In [220]:
# This negates only 43 sentences
nega = []
count = 0
for i in range(100):
    ret = Perturb.remove_negation(pdata[i])
    if ret == None:  # if nothing changes
        nega.append(pdata[i]) # append original sentence
    else: 
        nega.append(ret) # append negated paragraph
        count +=1
len(nega)

# Negation doesn't really change the sentiment labels, so they remain the same

100

In [221]:
count

44

In [222]:
"""# negate every sent in every para
neg = []
for i in range(100):
    ppdata = []
    
    # break down into sentences
    sentenced = sent_tokenize(str(no_punct[i]))
    pdata = list(nlp.pipe(sentenced))
    para = []
    # for each sentence in para
    for sent in pdata:
        ret = Perturb.remove_negation(sent)
        if ret == None:  # if nothing changes
            para.append(sent) # append original sentence
        else:
            para.append(ret[1]) # append negated sentence
    ppdata.append(para)
    neg.append(ppdata)
neg"""

'# negate every sent in every para\nneg = []\nfor i in range(100):\n    ppdata = []\n    \n    # break down into sentences\n    sentenced = sent_tokenize(str(no_punct[i]))\n    pdata = list(nlp.pipe(sentenced))\n    para = []\n    # for each sentence in para\n    for sent in pdata:\n        ret = Perturb.remove_negation(sent)\n        if ret == None:  # if nothing changes\n            para.append(sent) # append original sentence\n        else:\n            para.append(ret[1]) # append negated sentence\n    ppdata.append(para)\n    neg.append(ppdata)\nneg'

In [223]:
"""ret = Perturb.perturb(pdata, Perturb.remove_negation)
negated = []
for i in ret.data:
    negated.append(i[1])
negated[0]
len(negated)
pdata[0]"""

'ret = Perturb.perturb(pdata, Perturb.remove_negation)\nnegated = []\nfor i in ret.data:\n    negated.append(i[1])\nnegated[0]\nlen(negated)\npdata[0]'

# Make Final Output

In [248]:
output = []
for i in range(len(input_data)):
    dicti = {}
    dicti['reviewText'] = nega[i]
    dicti['sentiment'] = labels[i]
    dicti['category'] = "['typos', 'punct', 'negation']"
    output.append(dicti)
len(output)
print(output[0])

{'reviewText': Gotta listent o this! So craetive! Love his msuic - the words, the message! Some of my favorite songs ont his CD. I sohuld have bought it years ago, 'sentiment': 'positive', 'category': "['typos', 'punct', 'negation']"}


In [249]:
#with open('../data/predictions/dee_dump.json', 'w') as fout:
#    json.dump(output , fout)

In [246]:
test_json = [json.dumps(i) for i in output]
with open ('../data/predictions/dee_dump.json', 'w') as file:
    file.writelines(test_json)

TypeError: Object of type Doc is not JSON serializable